In [6]:
from datetime import datetime
print("Current Date/Time: ", datetime.now())

Current Date/Time:  2020-03-22 00:22:21.187723


In [1]:
!pip install torchsummary
!pip install torchvision

In [0]:
from torchsummary import summary
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Created folder EVA4 and s8 in EVA4. Upload the py files in to s8 
import sys
sys.path.append('/content/drive/My Drive/EVA4/s8')
!ls '/content/drive/My Drive/EVA4/s8'
import resnet18_arch
import data_loader
from train_test import Trainer

albumentation_data.py  GradCam.py   __pycache__       utils.py
cnn_model.py	       __init__.py  resnet18_arch.py
data_loader.py	       main.py	    train_test.py


<h3> Load Train & Test Data

In [5]:
train_loader, test_loader = data_loader.LoadData().load_data()

Files already downloaded and verified
Files already downloaded and verified


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(p=0.09) 
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=35, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(p=0.09) 
        )
        self.pool = nn.MaxPool2d(2, 2) 
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(p=0.09) 
        )
        self.conv1_1 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU()
        )
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(p=0.09) 
        )
        self.conv2_1 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=64, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU()
        )

        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=192, out_channels=256, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(p=0.09) 
        )
        self.conv3_1 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=64, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU()
        )
        self.conv4_1 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=64, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU()
        )
        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=192, out_channels=256, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(p=0.09) 
        )
        self.convblock9 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(p=0.09) 
        )
        self.convblock11 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(p=0.09) 
        )

        self.avg = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
        ) 
        self.conv6_1 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=10, kernel_size=(1, 1), padding=0, bias=False)
        )
        self.convblock_out = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
        ) # ou
        self.linear = nn.Linear(in_features=512,out_features=10, bias=False)

    def forward(self, x):
        x1 = x
        x2 = self.conv2(x1) #32 output
        x3 = self.conv3(torch.cat((x1,x2),dim=1)) #35 input channels 64 output channels
        x4 = self.pool(x3) #64 output channels
        x5 = self.convblock5(x4) #128 output channels 
        x5 = self.conv1_1(x5) #64 channels
        x6 = self.convblock6(torch.cat((x4 , x5),dim=1)) #64,64 input channels 256 output channels
        x6 = self.conv2_1(x6) #64 channels
        x7 = self.convblock7(torch.cat((x4, x5 , x6), dim=1)) #256 output channels
        x7 = self.conv3_1(x7) #64 output
        x8 = self.pool(torch.cat((x5 , x6 , x7),dim=1)) #64,64,64 input channels 192 output channels
        x9 = self.convblock8(x8) #192 input channels , 256 output
        x9 = self.conv4_1(x9) #64 output
        x10 = self.convblock9(torch.cat((x8 , x9),dim=1)) #256 input channels 256 output channels
        x11 = self.convblock11(torch.cat((x8 , x9 , x10),dim=1)) #192,64,256 == 512 inputchannels, output 256
        x12 = self.avg(x11) #1,1,512
        x12 = self.conv6_1(x12)
        x12 = x12.view(-1, 10)
        return F.log_softmax(x12)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          20,160
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
           Dropout-8           [-1, 64, 32, 32]               0
         MaxPool2d-9           [-1, 64, 16, 16]               0
           Conv2d-10          [-1, 128, 16, 16]          73,728
      BatchNorm2d-11          [-1, 128, 16, 16]             256
             ReLU-12          [-1, 128, 16, 16]               0
          Dropout-13          [-1, 128, 16, 16]               0
           Conv2d-14           [-1, 64,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [8]:
EPOCHS=10
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
for epoch in range(EPOCHS):
    print("EPOCH:", epoch+1, 'LR:', scheduler.get_lr()[0])
    Trainer().train(model, device, train_loader, optimizer,criterion)
    scheduler.step()
    Trainer().test(model, device, test_loader)

  0%|          | 0/391 [00:00<?, ?it/s]

EPOCH: 1 LR: 0.05


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Loss=1.0961114168167114 Batch_id=390 Accuracy=51.92: 100%|██████████| 391/391 [00:30<00:00, 12.82it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 1.1448, Accuracy: 5913/10000 (59.13%)

EPOCH: 2 LR: 0.05


Loss=0.6962776184082031 Batch_id=390 Accuracy=70.23: 100%|██████████| 391/391 [00:30<00:00, 12.72it/s]
  0%|          | 0/391 [00:00<?, ?it/s]

...


Test set: Average loss: 0.3993, Accuracy: 8641/10000 (86.41%)

EPOCH: 9 LR: 0.005000000000000001


Loss=0.4016169607639313 Batch_id=390 Accuracy=89.76: 100%|██████████| 391/391 [00:30<00:00, 12.67it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 0.3886, Accuracy: 8669/10000 (86.69%)

EPOCH: 10 LR: 0.005000000000000001


Loss=0.1759653389453888 Batch_id=390 Accuracy=90.08: 100%|██████████| 391/391 [00:30<00:00, 12.73it/s]



Test set: Average loss: 0.3996, Accuracy: 8646/10000 (86.46%)

